# Connors Reversal Strategy Batch Optimization

这个notebook用于对多个交易对进行Connors Reversal策略的批量优化。

In [1]:
# 导入必要的库
import os
import pandas as pd
from datetime import datetime
from glob import glob
import numpy as np
from backtesting import Backtest
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output  # 新增：导入 clear_output

In [2]:
# 在文件开头，导入库之后添加全局配置
from ConnorsReversalShort import ConnorsReversalShort

CONFIG = {
    # 策略相关配置
    'strategy': {
        'class': ConnorsReversalShort,
        'name': ConnorsReversalShort.__name__
    },
    
    # 数据相关配置
    'data_path': r'\\znas\Main\futures',
    'start_date': '2024-01-01',
    'end_date': '2025-02-08',
    'source_timeframe': '1m',
    'target_timeframes': ['1H', '30min', '15min', '5min', '1min'],
    
    # 文件保存配置
    'reports_path': 'reports',
    'results_filename_template': 'optimization_results_{strategy_name}_{start_date}-{end_date}.xlsx',
    
    # 回测参数配置
    'commission': 0.0004,
    'margin': 1/2,
    'initial_capital': 10000,
    'trade_on_close': True,
    'exclusive_orders': True,
    'hedging': False,
    
    # 优化参数配置
    'optimization_params': {
        'highest_point_bars': range(5, 51, 5),    # 慢速RSI均线使用的最高点柱数：5, 10, 15, 20, 25, 30, 35, 40, 45, 50
        'rsi_length': range(2, 51, 5),             # RSI指标计算周期：2, 7, 12, 17, 22, 27, 32, 37, 42, 47
        'buy_barrier': range(65, 86, 5),           # 买入障碍水平：65, 70, 75, 80, 85
        'dca_parts': range(4, 12, 1)               # 加仓部分数量：4 到 11
    },
    
    # 优化方法配置
    'optimization_settings': {
        'method': 'sambo',
        'max_tries': 200,
        'random_state': 0
    },
    
    # 自定义评分函数权重配置
    'score_weights': {
        'ret_weight': 0.6,        # 收益率权重
        'sqn_weight': 0.4,        # SQN权重
        'sharpe_weight': 0.2,     # 夏普比率权重
        'win_rate_weight': 0.15,  # 胜率权重
        'dd_weight': 0.1,         # 回撤惩罚权重
        'min_trades': 50          # 最小交易次数要求
    }
}

In [3]:
def load_and_resample_data(symbol, start_date, end_date, source_timeframe='1m', target_timeframe='30min', data_path=r'\\znas\Main\futures'):
    """
    加载并重采样期货数据
    """
    # 生成日期范围
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # 准备存储所有数据的列表
    all_data = []
    
    # 标准化交易对名称
    formatted_symbol = symbol.replace('/', '_').replace(':', '_')
    if not formatted_symbol.endswith('USDT'):
        formatted_symbol = f"{formatted_symbol}USDT"
    
    # 遍历每一天
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        # 构建文件路径
        file_path = os.path.join(data_path, date_str, f"{date_str}_{formatted_symbol}_USDT_{source_timeframe}.csv")
        
        try:
            if os.path.exists(file_path):
                # 读取数据
                df = pd.read_csv(file_path)
                df['datetime'] = pd.to_datetime(df['datetime'])
                all_data.append(df)
            else:
                print(f"文件不存在: {file_path}")
        except Exception as e:
            print(f"读取文件出错 {file_path}: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError(f"未找到 {symbol} 在指定日期范围内的数据")
    
    # 合并所有数据
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values('datetime')
    
    # 设置时间索引
    combined_df.set_index('datetime', inplace=True)
    
    # 重采样到目标时间周期
    resampled = combined_df.resample(target_timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()  # 立即删除NaN值
    
    # 转换为backtesting.py格式
    backtesting_df = pd.DataFrame({
        'Open': resampled['open'],
        'High': resampled['high'],
        'Low': resampled['low'],
        'Close': resampled['close'],
        'Volume': resampled['volume']
    })
    
    # 确保所有数据都是数值类型并删除任何无效值
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        backtesting_df[col] = pd.to_numeric(backtesting_df[col], errors='coerce')
    
    # 最终清理
    backtesting_df = backtesting_df.dropna()
    
    return backtesting_df

In [4]:
def get_all_symbols(data_path, date_str):
    """获取指定日期目录下的所有交易对"""
    daily_path = os.path.join(data_path, date_str)
    if not os.path.exists(daily_path):
        return []
    
    files = glob(os.path.join(daily_path, f"{date_str}_*_USDT_1m.csv"))
    symbols = []
    for file in files:
        # 从文件名提取交易对名称
        filename = os.path.basename(file)
        symbol = filename.split('_')[1]
        if symbol not in symbols:
            symbols.append(symbol)
    return symbols

def verify_data_completeness(symbol, start_date, end_date, data_path):
    """验证交易对在指定时间段内的数据完整性"""
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        file_path = os.path.join(data_path, date_str, f"{date_str}_{symbol}_USDT_1m.csv")
        if not os.path.exists(file_path):
            return False
    return True

In [5]:
def optimize_strategy(symbol, start_date, end_date, data_path, results_file, 
                     strategy_cls, optimization_params, source_timeframe, target_timeframe,
                     backtest_config, optimization_settings, custom_score_fn):
    """对单个交易对进行策略优化并及时保存结果"""
    try:
        print(f"开始加载 {symbol} 的数据...")
        df = load_and_resample_data(
            symbol=symbol,
            start_date=start_date,
            end_date=end_date,
            source_timeframe=source_timeframe,
            target_timeframe=target_timeframe,
            data_path=data_path
        )
        
        bt = Backtest(
            df,
            strategy_cls,
            **backtest_config
        )
        
        print("\n开始参数优化...")
        stats, heatmap, optimize_result = bt.optimize(
            **optimization_params,
            constraint=lambda p: True,
            maximize=custom_score_fn,
            **optimization_settings,
            return_heatmap=True,
            return_optimization=True
        )
        
        clear_output(wait=True)  # 清除当前 cell 的所有输出
        
        # 获取排序后的前5个最优结果
        sorted_results = heatmap.sort_values(ascending=False).head(5)
        results = []
        
        for rank, (params, score) in enumerate(sorted_results.items(), 1):
            # 将参数元组转换为字典
            current_params = dict(zip(optimization_params.keys(), params))
            
            # 使用当前参数运行回测
            final_stats = bt.run(**current_params)
            
            result = {
                'Symbol': symbol,
                'Rank': rank,  # 1-5的排名
                'Initial Capital': 10000,
                'Score': score,  # 使用当前参数组合的得分
                **current_params,
                'Start': final_stats['Start'],
                'End': final_stats['End'],
                'Duration': final_stats['Duration'],
                'Exposure Time [%]': final_stats['Exposure Time [%]'],
                'Equity Final [$]': final_stats['Equity Final [$]'],
                'Equity Peak [$]': final_stats['Equity Peak [$]'],
                'Total Return (%)': final_stats['Return [%]'],
                'Buy & Hold Return (%)': final_stats['Buy & Hold Return [%]'],
                'Return (Ann.) (%)': final_stats['Return (Ann.) [%]'],
                'Volatility (Ann.) (%)': final_stats['Volatility (Ann.) [%]'],
                'CAGR (%)': final_stats.get('CAGR [%]', None),
                'Sharpe Ratio': final_stats['Sharpe Ratio'],
                'Sortino Ratio': final_stats.get('Sortino Ratio', None),
                'Calmar Ratio': final_stats.get('Calmar Ratio', None),
                'Max Drawdown (%)': final_stats['Max. Drawdown [%]'],
                'Avg Drawdown (%)': final_stats.get('Avg. Drawdown [%]', None),
                'Max Drawdown Duration': final_stats.get('Max. Drawdown Duration', None),
                'Avg Drawdown Duration': final_stats.get('Avg Drawdown Duration', None),
                'Total Trades': final_stats['# Trades'],
                'Win Rate (%)': final_stats['Win Rate [%]'],
                'Best Trade (%)': final_stats.get('Best Trade [%]', None),
                'Worst Trade (%)': final_stats.get('Worst Trade [%]', None),
                'Avg Trade (%)': final_stats.get('Avg. Trade [%]', None),
                'Max Trade Duration': final_stats.get('Max. Trade Duration', None),
                'Avg Trade Duration': final_stats.get('Avg. Trade Duration', None),
                'Profit Factor': final_stats.get('Profit Factor', None),
                'Expectancy (%)': final_stats.get('Expectancy [%]', None),
                'SQN': final_stats.get('SQN', None),
                'Kelly Criterion': final_stats.get('Kelly Criterion', None),
                'Target Timeframe': target_timeframe,
                'Optimization Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            results.append(result)
        
        print(f"完成前{len(results)}个最佳结果的处理")
        
        # 保存结果到Excel文件
        try:
            existing_df = pd.read_excel(results_file)
            new_df = pd.concat([existing_df, pd.DataFrame(results)], ignore_index=True)
        except FileNotFoundError:
            new_df = pd.DataFrame(results)
        
        new_df.to_excel(results_file, index=False)
        print(f"成功保存 {len(results)} 个优化结果")
        return results
        
    except Exception as e:
        print(f"优化 {symbol} 时出错: {str(e)}")
        print(f"错误类型: {type(e).__name__}")
        print(f"错误详情: {str(e)}")
        import traceback
        print("完整错误追踪:")
        print(traceback.format_exc())
        return None


In [6]:
def main():
    config = CONFIG
    
    def custom_score(stats):
        """自定义评分函数，使用config中的权重配置"""
        weights = config['score_weights']
        
        sharpe = stats['Sharpe Ratio']
        max_dd = stats['Max. Drawdown [%]']
        ret = stats['Return [%]']
        win_rate = stats['Win Rate [%]']
        sqn = stats['SQN']
        trades = stats['# Trades']

        dd_penalty = 1 / (1 + abs(max_dd/100))
        trade_penalty = 1 if trades >= weights['min_trades'] else trades / weights['min_trades']

        score = (
            weights['ret_weight'] * (ret/100) +
            weights['sqn_weight'] * sqn +
            weights['sharpe_weight'] * sharpe +
            weights['win_rate_weight'] * (win_rate/100) +
            weights['dd_weight'] * dd_penalty
        ) * trade_penalty

        return score

    # 确保报告目录存在
    os.makedirs(config['reports_path'], exist_ok=True)
    
    # 使用配置中的开始日期获取交易对列表
    start_date_obj = datetime.strptime(config['start_date'], '%Y-%m-%d')
    symbols = get_all_symbols(config['data_path'], start_date_obj.strftime('%Y-%m-%d'))
    print(f"总共找到 {len(symbols)} 个交易对:")
    
    # 构造结果文件名
    start_clean = config['start_date'].replace("-", "")
    end_clean = config['end_date'].replace("-", "")
    master_file = os.path.join(
        config['reports_path'], 
        config['results_filename_template'].format(
            strategy_name=config['strategy']['name'],
            start_date=start_clean,
            end_date=end_clean
        )
    )
    
    print(f"使用数据路径: {config['data_path']}")
    print(f"回测结果将保存到: {master_file}")
    print(f"回测时间范围: {config['start_date']} 到 {config['end_date']}")
    
    # 修改检查逻辑
    try:
        global_df = pd.read_excel(master_file)
        if not global_df.empty:
            if 'Target Timeframe' in global_df.columns and 'Symbol' in global_df.columns and 'Rank' in global_df.columns:
                # 对每个Symbol-Timeframe组合，检查是否有完整的5个排名
                optimized_combinations = set()
                for symbol in global_df['Symbol'].unique():
                    for tf in global_df['Target Timeframe'].unique():
                        ranks = global_df[(global_df['Symbol'] == symbol) & 
                                       (global_df['Target Timeframe'] == tf)]['Rank'].tolist()
                        # 只有当存在完整的5个排名时，才认为这个组合已经完成优化
                        if len(ranks) == 5 and set(ranks) == set(range(1, 6)):
                            optimized_combinations.add((symbol, tf))
                
                if optimized_combinations:
                    combos_str = ", ".join([f"{s}-{tf}" for s, tf in optimized_combinations])
                    print(f"已完成优化的组合: {combos_str}")
            else:
                optimized_combinations = set()
                print("警告: 结果文件缺少必要的列，将重新开始优化")
        else:
            optimized_combinations = set()
            print("优化结果文件为空")
    except FileNotFoundError:
        global_df = pd.DataFrame()
        optimized_combinations = set()
        print("未找到优化结果文件，将新建文件")
    
    # 统计所有需要优化的组合
    total_combinations = [(s, tf) for s in symbols for tf in config['target_timeframes']]
    remaining_combinations = [combo for combo in total_combinations if combo not in optimized_combinations]
    print(f"\n剩余需要优化的组合数量: {len(remaining_combinations)} 个")
    
    # 针对每个交易对进行优化
    for i, symbol in enumerate(symbols, 1):
        print(f"\n正在处理第 {i}/{len(symbols)} 个交易对: {symbol}")
        print("-" * 50)
        
        # 验证数据完整性
        print(f"验证 {symbol} 的数据完整性...")
        if not verify_data_completeness(symbol, config['start_date'], config['end_date'], config['data_path']):
            print(f"警告: 跳过 {symbol} - 数据不完整")
            continue
        print(f"{symbol} 数据完整性验证通过")
        
        # 针对每个目标时间周期进行优化
        for tf in config['target_timeframes']:
            # 检查当前组合是否已经完成优化
            if (symbol, tf) in optimized_combinations:
                print(f"组合 {symbol}-{tf} 已完成优化（存在5个排名），跳过")
                continue
            
            print(f"\n开始针对时间周期 {tf} 优化 {symbol} ...")
            results = optimize_strategy(
                symbol=symbol,
                start_date=config['start_date'],
                end_date=config['end_date'],
                data_path=config['data_path'],
                results_file=master_file,
                strategy_cls=config['strategy']['class'],
                optimization_params=config['optimization_params'],
                source_timeframe=config['source_timeframe'],
                target_timeframe=tf,
                backtest_config={
                    'commission': config['commission'],
                    'margin': config['margin'],
                    'trade_on_close': config['trade_on_close'],
                    'exclusive_orders': config['exclusive_orders'],
                    'hedging': config['hedging']
                },
                optimization_settings=config['optimization_settings'],
                custom_score_fn=custom_score
            )
            
            if results:
                print(f"完成 {symbol} 在 {tf} 时间周期下的优化")
            else:
                print(f"警告: {symbol} 在 {tf} 时间周期下优化失败")


In [ ]:

if __name__ == '__main__':
    main()


完成前5个最佳结果的处理
成功保存 5 个优化结果
完成 1000FLOKIUSDT 在 5min 时间周期下的优化

开始针对时间周期 1min 优化 1000FLOKIUSDT ...
开始加载 1000FLOKIUSDT 的数据...

开始参数优化...
['c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\python310.zip', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\DLLs', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting', '', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages\\win32', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages\\win32\\lib', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages\\Pythonwin']


Backtest.optimize:   0%|          | 0/200 [00:00<?, ?it/s]